In [1]:
from iracingdataapi.client import irDataClient 
import pandas as pd
import numpy as np 
import datetime as dt
idc = irDataClient(username = 'YOUR IRACING USERNAME', password = 'YOUR IRACING PASSWORD')

In [2]:
# Function for getting the season ID of the season of interest given a year, quarter, and name 

def seasonID(year, quarter, name):
    # For name input full name of series as seen in iracing client, in all lowercase
    raw = irDataClient.season_list(idc, season_year=year, season_quarter=quarter)
    dict_list = raw['seasons']
    for dict in dict_list:
        if name in dict['series_name'].lower():
            return dict['season_id'], dict['season_name']
        
Formula4Q423 = seasonID(2023, 4, 'formula 4')

In [6]:
# Uses irDataClient to pull data of interest for every subsession of a given season from the iracing data api 

def season_sessions(season_id):
    full_season_dict = []
    for i in range(0, 12):
        raw_week = irDataClient.result_season_results(idc, season_id, event_type = 5, race_week_num = i)
                
        season_dict = {
            'subsession_ID': [result['subsession_id'] for result in raw_week['results_list']],
            'track_name': [result['track']['track_name'] for result in raw_week['results_list']],
            'track_id': [result['track']['track_id'] for result in raw_week['results_list']],
            'driver_count': [result['num_drivers'] for result in raw_week['results_list']],
            'week_num': [result['race_week_num'] for result in raw_week['results_list']],
            'date': [result['start_time'] for result in raw_week['results_list']],
            'SOF': [result['event_strength_of_field'] for result in raw_week['results_list']],
            'lead_changes': [result['num_lead_changes'] for result in raw_week['results_list']]
        }
        full_season_dict.append(pd.DataFrame(season_dict))
    season_df = pd.concat(full_season_dict, ignore_index=True)
    return season_df 


In [4]:
# Create the dataframe containing data for every subsession in a given season 

fiaF4S423 = season_sessions(4411)

In [4]:
def sub_avg_inc(subsession):
    # Returns the average incident count of a given subsession 
    result_subsession = irDataClient.result(idc, subsession)
    result_race_dict = result_subsession['session_results'][2]['results']
    subsession_incidents = [result['incidents'] for result in result_race_dict]
    avg_inc = np.mean(subsession_incidents)
    return avg_inc

In [5]:
def sub_avg_license(subsession):
    # Returns the average license level of a given subsession
    result_subsession = irDataClient.result(idc, subsession)
    result_race_dict = result_subsession['session_results'][2]['results']
    subsession_license_level = [result['old_license_level'] for result in result_race_dict]
    avg_license_level = np.mean(subsession_license_level)
    return avg_license_level

In [14]:
def get_temperature(subsession):
    # Returns the temperature of a given subsession
    result_subsession = irDataClient.result(idc, subsession)
    temperature = result_subsession['weather']['temp_value']
    return temperature

In [6]:
# Create new column containing the average incident count of a given subsession

fiaF4S423['average_incidents'] = [sub_avg_inc(session) for session in fiaF4S423['subsession_ID']]

In [ ]:
# Create new column contianing the average license level of a given subsession

fiaF4S423['average_license'] = [sub_avg_license(session) for session in fiaF4S423['subsession_ID']]

In [15]:
# Create new column containing the temperature of a given subsession

fiaF4S423['temperature'] = [get_temperature(subsession) for subsession in fiaF4S423['subsession_ID']]

In [14]:
# Get the dictionaries from irDataClient.get_tracks() only for tracks raced in the season of interest

unique_tracks = list(fiaF4S423['track_id'].unique())
track_list = irDataClient.get_tracks(idc)
relevant_tracks = [track for track in track_list if track['track_id'] in unique_tracks]

In [28]:
# Pull track characteristics from relevant_tracks, merge with original dataframe on track_id

track_characteristics = pd.DataFrame(
    {
        'track_id': [track['track_id'] for track in relevant_tracks],
        'track_length': [track['track_config_length'] for track in relevant_tracks],
        'track_corners': [track['corners_per_lap'] for track in relevant_tracks],
        'free_track': [track['free_with_subscription'] for track in relevant_tracks]
    }
)

fiaF4S423_merged = fiaF4S423.merge(track_characteristics, how = 'inner', on = 'track_id')

In [49]:
# Convert to datetime and create column for day of the week

fiaF4S423_merged['date'] = pd.to_datetime(fiaF4S423_merged['date'])
fiaF4S423_merged['day_of_week'] = fiaF4S423_merged['date'].dt.dayofweek

In [55]:
# Export the final csv 

fiaF4S423_merged.to_csv(r'C:\Users\Bennett\Documents\IRacing_Incident_Prediction\CSVs\f4_df_1.csv')